## Imports & Configuration

In [1]:
import yaml
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
from sqlalchemy import create_engine
import sys
sys.path.append('./src')

In [ ]:
# install if needed
# %pip install pyyaml
# %pip install sqlalchemy
# %pip install pymysql
# %pip freeze > requirements.txt

In [2]:
def load_config(file_path):
  try:
    with open(file_path, 'r') as file:
      config = yaml.safe_load(file)
    return config
  except FileNotFoundError:
    print(f"Die Datei {file_path} wurde nicht gefunden.")
    return None
  except yaml.YAMLError as e:
    print(f"Fehler beim Lesen der YAML-Datei: {e}")
    return None

# Konfigurationsdatei laden
config = load_config('../config.yml')

In [3]:
# Enter the city names for which you want information
# -------------------------------------------------------------------------------
# Example: list_of_cities = ['Hamburg', 'Munich', 'London', 'New_York_City', 'Los_Angeles', 'Tokyo', 'Dubai', 'Bangkok', 'Wellington']
# Example 2 (m:n airports) list_of_cities = ['Basel', 'Mulhouse', 'Freiburg']
# -------------------------------------------------------------------------------

# list_of_cities = ['Berlin','Hamburg', 'Munich', 'London', 'New_York_City', 'Los_Angeles', 'Tokyo', 'Dubai', 'Bangkok', 'Wellington']
list_of_cities = ['Berlin', 'London', 'New_York_City']

## Webscraping

In [4]:
def get_city_data(city):

  url = f'https://en.wikipedia.org/wiki/{city}'
  r = requests.get(url)
  soup = BeautifulSoup(r.content, 'html.parser')
  # 1. Extracting Data
  city_dict = {}
  city_dict['city'] = soup.select_one('.firstHeading').get_text() if soup.select_one(".firstHeading") else city
  city_dict['country'] = soup.select(".infobox-data")[0].get_text() if len(soup.select(".infobox-data")) else "N/A"
  city_dict['latitude'] = soup.select_one(".latitude").get_text() if soup.select_one(".latitude") else "N/A"
  city_dict['longitude'] = soup.select_one(".longitude").get_text() if soup.select_one(".longitude") else "N/A"

  # Extract and process population data
  population_header = soup.select_one('th.infobox-header:-soup-contains("Population")')
  if population_header:
    population_data = population_header.parent.find_next_sibling()
    city_dict['population'] = population_data.find(string=re.compile(r'\d+')) if population_data else "N/A"

    population_year_div = population_header.find("div", class_="ib-settlement-fn")
    if population_year_div:
      for sup in population_year_div.find_all("sup"):
        sup.decompose() # Remove sup elements and extract year  
      year_str = population_year_div.get_text(strip=True).strip("()") # Extract text without brackets
      city_dict['population_year'] = extract_year(year_str)
    else:
      city_dict['population_year'] = 0
  else:
    city_dict['population'] = 0
    city_dict['population_year'] = 0
  return city_dict

def extract_year(value):
  match = re.search(r'\b(\d{4,})\b', value)
  if match:
    year = int(match.group(1))
    if 1900 <= year <= 2100:
      return year
  return None

# Function to convert coordinates from degrees/minutes/seconds to decimal degrees
def dms_to_decimal(coord):
  match = re.match(r"(\d+)°(\d+)?′?(\d+)?(?:″)?([NSEW])", coord)
  if not match:
      return None
  
  degrees = int(match.group(1))
  minutes = int(match.group(2) or 0)
  seconds = int(match.group(3) or 0)
  direction = match.group(4)
  decimal = degrees + (minutes / 60) + (seconds / 3600)

  if direction in ['S', 'W']:
      decimal = -decimal
  return decimal

def webscraping_function(cities):
  city_list = [] # Initializing an Empty List

  for city in cities: # Looping Through Cities
    city_data = get_city_data(city)
    city_list.append(city_data) # Appending Data to List
  cities_df = pd.DataFrame(city_list) # Creating a DataFrame
  return cities_df

# Using the webscraping_function to get Cities information
cities_df = webscraping_function(list_of_cities)

# Data Cleaning
# ----------------------------------------------------------------
# Apply dms_to_decimal function to columns
cities_df['latitude'] = cities_df['latitude'].apply(dms_to_decimal)
cities_df['longitude'] = cities_df['longitude'].apply(dms_to_decimal)

# Set column population to numeric
# cities_df['population'] = cities_df['population'].str.replace(',', '').astype(int)
cities_df['population'] = pd.to_numeric(cities_df['population'].str.replace(',', ''), errors='coerce')

# Replace '\xa0' in column 'country'
cities_df['country'] = cities_df['country'].str.replace('\xa0', '', regex=False)

# cities_df.info()
cities_df

,city,country,latitude,longitude,population,population_year
0,Berlin,Germany,52.520000,13.405000,3596999,2022
1,London,United Kingdom,51.507222,-0.127500,8866180,2022
2,New York City,United States,40.712778,-74.006111,8804190,2020


## Openweathermap.org API

The data can be found in the list `weather_json['list']`.<br>
Each element of this list is a dictionary containing the weather forecast for a specific time interval. At the end there is a separate **city dictionary** with information about the city.<br>
To maintain an overview, we first extract all the keys from a single forecast data set (`weather_json['list'][0]`) that we are interested in and then look at the structure of `weather_json['city']`.

In [5]:
# Functions for request the openweathermap.org API using city names from cities_df

# API-Config
api_config = config.get("apis", {}).get("openweathermap", {})
APIkey = api_config.get("api_key", "")
base_url = api_config.get("base_url", "")

# Convert UTC timestamp to local time based on time zone offset (e.g. +01:00)
def convert_to_local_time(timestamp, timezone_offset):
	utc_time = datetime.fromtimestamp(timestamp, tz=timezone.utc)  # Timezones UTC-times
	local_time = utc_time + timedelta(seconds=timezone_offset)  # Local time with offset
	return local_time

#def convert_to_local_time(timestamp, timezone_offset, city_timezone):
#    utc_time = datetime.fromtimestamp(timestamp, tz=dt_timezone.utc)
#    # Zeitzone der Stadt zuweisen
#    if city_timezone:
#        local_time = utc_time.astimezone(ZoneInfo(city_timezone))
#    else:
#        local_time = utc_time + timedelta(seconds=timezone_offset)
#    return local_time

def get_request_timestamp(city_timezone):
    """
    Gibt die aktuelle Zeit mit der Zeitzone der Stadt zurück.
    """
    utc_now = datetime.now(timezone.utc)
    try:
        # Zeitzone der Stadt anwenden
        return utc_now.replace(tzinfo=ZoneInfo("UTC")).astimezone(ZoneInfo(city_timezone))
    except Exception as e:
        print(f"Fehler beim Ermitteln der Zeitzone: {e}")
        return utc_now  # Fallback: UTC-Zeit

# Function for querying weather data
def get_weather_data(cities_df):
	weather_infos = []
	cities = cities_df['city'].dropna().unique()

	for city in cities:
		# API-Request
		params = {'q': city, 'appid': APIkey, 'units': 'metric'}
		response = requests.get(base_url, params=params)
		if response.status_code != 200:
			print(f"Error when retrieving data for {city}: {response.status_code}")
			continue
		
		weather_json = response.json()
		
		# Extract city information from json 
		city_info = weather_json.get('city', {})
		city_name = city_info.get('name', city)
		sunrise = city_info.get('sunrise')
		sunset = city_info.get('sunset')
		timezone_offset = city_info.get('timezone')  # Offset is in format seconds

		# Convert sunrise and sunset to local time using convert_to_local_time function
		if sunrise and sunset:
			sunrise_local = convert_to_local_time(sunrise, timezone_offset)
			sunset_local = convert_to_local_time(sunset, timezone_offset)
		else:
			sunrise_local, sunset_local = None, None

		# Define a dynamical timezones
		city_timezones = {
    	"Berlin": "Europe/Berlin",
    	"New York": "America/New_York",
    	"Tokyo": "Asia/Tokyo"
			}

		# Process the forecast
		forecasts = weather_json.get('list', [])
		for forecast in forecasts:
			forecast_time_utc = datetime.strptime(forecast.get('dt_txt'), "%Y-%m-%d %H:%M:%S")
			forecast_time_local = forecast_time_utc + timedelta(seconds=timezone_offset)
			# city_timezone = city_timezones.get(city, "UTC")  # Fallback auf UTC, falls nicht gefunden
			# forecast_time_local = forecast_time_utc.astimezone(ZoneInfo(city_timezone))
			
			
			weather_infos.append({
				'city': city_name,
				'forecast_time': forecast_time_local.strftime("%Y-%m-%d %H:%M:%S"),
				'sunrise': sunrise_local.strftime("%Y-%m-%d %H:%M:%S") if sunrise_local else None,
				'sunset': sunset_local.strftime("%Y-%m-%d %H:%M:%S") if sunset_local else None,
				'temperature': forecast.get('main', {}).get('temp'),
				'feels_like': forecast.get('main', {}).get('feels_like'),
				'humidity': forecast.get('main', {}).get('humidity'),
				'weather_description': (forecast.get('weather', [{}])[0]).get('description'),
				'wind_speed': forecast.get('wind', {}).get('speed'),
				'wind_direction': forecast.get('wind', {}).get('deg'),
				'rain': forecast.get('rain', {}).get('3h', 0),
				# 'request_time': request_time.strftime("%Y-%m-%d %H:%M:%S %z"),  # Lokale Abfragezeit

			})
	
	# Create DataFrame
	return pd.DataFrame(weather_infos)

# Calling up the function and processing the data
weather_df = get_weather_data(cities_df)
weather_df['forecast_time'] = pd.to_datetime(weather_df['forecast_time'])
weather_df['sunrise'] = pd.to_datetime(weather_df['sunrise'])
weather_df['sunset'] = pd.to_datetime(weather_df['forecast_time'])
weather_df

,city,forecast_time,sunrise,sunset,temperature,feels_like,humidity,weather_description,wind_speed,wind_direction,rain
0,Berlin,2025-01-25 13:00:00,2025-01-25 07:58:24,2025-01-25 13:00:00,10.08,9.15,77,clear sky,7.30,219,0.0
1,Berlin,2025-01-25 16:00:00,2025-01-25 07:58:24,2025-01-25 16:00:00,10.86,9.86,71,scattered clouds,5.62,227,0.0
2,Berlin,2025-01-25 19:00:00,2025-01-25 07:58:24,2025-01-25 19:00:00,10.49,9.55,75,broken clouds,4.10,224,0.0
3,Berlin,2025-01-25 22:00:00,2025-01-25 07:58:24,2025-01-25 22:00:00,8.74,7.97,91,overcast clouds,1.73,313,0.0
4,Berlin,2025-01-26 01:00:00,2025-01-25 07:58:24,2025-01-26 01:00:00,7.60,6.57,95,light rain,1.82,53,0.4
...,...,...,...,...,...,...,...,...,...,...,...
115,New York,2025-01-29 16:00:00,2025-01-25 07:11:51,2025-01-29 16:00:00,4.50,-0.20,74,overcast clouds,7.18,248,0.0
116,New York,2025-01-29 19:00:00,2025-01-25 07:11:51,2025-01-29 19:00:00,2.91,-2.53,73,broken clouds,7.95,262,0.0
117,New York,2025-01-29 22:00:00,2025-01-25 07:11:51,2025-01-29 22:00:00,0.71,-4.90,56,clear sky,6.76,289,0.0
118,New York,2025-01-30 01:00:00,2025-01-25 07:11:51,2025-01-30 01:00:00,-0.45,-6.54,59,clear sky,7.10,293,0.0


## Airport & Flight Data

### Get Airport information

In [6]:
# Define a radius in which airports are to be displayed around the cities
# -------------------------------------------------------------------------
radius = 35

# What is the maximum number of airports you want to display
# -------------------------------------------------------------------------
limit = 10

In [7]:
def get_airports(cities_df):

  # Load AeroDataBox API data from the configuration file
  aerodatabox_config = config.get("apis", {}).get("aerodatabox", {}).get("airports", {})
  url = aerodatabox_config.get("url", "")
  headers = aerodatabox_config.get("headers", {})

  def get_airports_from_coords(lat, lon):
    querystring = {
      "lat": lat,
      "lon": lon,
      "radiusKm": radius,
      "limit": limit,
      "withFlightInfoOnly": "true"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
      airports = response.json().get('items', [])
      return pd.json_normalize(airports) if airports else pd.DataFrame()
    else:
      print(f"API request failed for {lat}, {lon} with status code: {response.status_code}")
      return pd.DataFrame()

  # Group cities by unique coordinates
  unique_coords = cities_df[['latitude', 'longitude', 'city']].drop_duplicates()

  # Call up airport information
  all_airports = []
  for _, row in unique_coords.iterrows():
      lat, lon, city = row['latitude'], row['longitude'], row['city']
      airport_data = get_airports_from_coords(lat, lon)
      if not airport_data.empty:
          airport_data['city'] = city
          all_airports.append(airport_data)

  # Merge results
  return pd.concat(all_airports, ignore_index=True) if all_airports else pd.DataFrame()

airports_df = get_airports(cities_df)
airports_df

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon,city
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889,Berlin
1,EGLC,LCY,London City,City,London,GB,Europe/London,51.50530,0.055277,London
2,EGLL,LHR,London Heathrow,Heathrow,London,GB,Europe/London,51.47060,-0.461941,London
3,EGKR,KRH,Redhill Aerodrome,Aerodrome,Redhill,GB,Europe/London,51.21360,-0.138611,London
4,KLGA,LGA,New York La Guardia,La Guardia,New York,US,America/New_York,40.77720,-73.872600,New York City
5,KEWR,EWR,Newark Liberty,Liberty,Newark,US,America/New_York,40.69250,-74.168700,New York City
6,KTEB,TEB,Teterboro,Teterboro,Teterboro,US,America/New_York,40.85010,-74.060800,New York City
7,KJFK,JFK,New York John F Kennedy,John F Kennedy,New York,US,America/New_York,40.63980,-73.778900,New York City
8,KCDW,CDW,Caldwell Essex County,Essex County,Caldwell,US,America/New_York,40.87520,-74.281400,New York City


### Get Flight information

In [8]:
# Function for arrival flights for the next day
def get_querystring_for_nextday():

  # From current date & time, set to the next day 00:00 and end time to max. 12 hours after start time
  now = datetime.now()
  next_day = now + timedelta(days=1)
  start_time = next_day.replace(hour=0, minute=0, second=0, microsecond=0)
  end_time = start_time + timedelta(hours=12)
  
  # Querystring for API
  querystring = {
    "withLeg": "true",
    "direction": "Arrival",
    "withCancelled": "false",
    "withCodeshared": "true",
    "withCargo": "false",
    "withPrivate": "false",
    #"withLocation": "false", # ???
    "start": start_time.strftime('%Y-%m-%dT%H:%M'),  # Start time for the next day ('YYYY-MM-DDTHH:mm')
    "end": end_time.strftime('%Y-%m-%dT%H:%M')       # End time for the next day ('YYYY-MM-DDTHH:mm')
  }
  return querystring

# Retrieves flight data for an airport
def get_flights(airport_code, querystring):
	
	# Load configuration for the AeroDataBox API & Check configuration
	aerodatabox_config = config.get("apis", {}).get("aerodatabox", {}).get("flights", {})
	url_template = aerodatabox_config.get("url", "")
	headers = aerodatabox_config.get("headers", {})

	if not url_template or not headers:
		print("Error: Invalid API configuration.")
		return None

	# Assemble API URL & send request
	url = url_template.format(airport_code=airport_code, start=querystring["start"], end=querystring["end"])
	response = requests.get(url, headers=headers, params=querystring)

	if response.status_code == 200:
		return response.json()
	elif response.status_code == 204:
		print(f"No flights for Airport: {airport_code}. Skipped.")
		return None
	else:
		print(f"Error {response.status_code} for Airport: {airport_code}")
		return None

# Converts the flight data into a DataFrame
def flights_to_dataframe(flights_data):
	if not flights_data or 'arrivals' not in flights_data:
		print("No flight data available.")
		return None

	# Extract arrivals and convert to list
	arrivals = flights_data['arrivals']
	flight_list = [{
		"FlightNumber": flight.get('number', 'N/A'),
		"Airline": flight.get('airline', {}).get('name', 'N/A'),
		"Aircraft": flight.get('aircraft', {}).get('model', 'N/A'),
		"OriginAirport": flight.get('departure', {}).get('airport', {}).get('name', 'N/A'),
		"OriginIATA": flight.get('departure', {}).get('airport', {}).get('iata', 'N/A'),
		"Terminal": flight.get('arrival', {}).get('terminal', 'N/A'),
		"ScheduledArrival": flight.get('arrival', {}).get('scheduledTime', {}).get('local', 'N/A'),
		"ActualArrival": flight.get('arrival', {}).get('revisedTime', {}).get('local', 'N/A'),
		"Note": flight.get('status', 'N/A'),
		}
		for flight in arrivals
	]

	return pd.DataFrame(flight_list)

# Retrieves flight data for all airports and creates a combined DataFrame
def get_flight_data_for_airports(airports_df):
	
	all_flights_df = []

	for _, row in airports_df.iterrows():
		airport_code = row['iata']  # Use IATA-Code of airports
		querystring = get_querystring_for_nextday()  # Querystring for next day flights
		flights_data = get_flights(airport_code, querystring)  # Request flight data

		# Convert flight data to DataFrame
		if flights_data:
			flights_df = flights_to_dataframe(flights_data)
			if flights_df is not None and not flights_df.empty:
				flights_df['AirportName'] = airport_code  # Assign airport
				all_flights_df.append(flights_df)

	# Return combined results
	return pd.concat(all_flights_df, ignore_index=True) if all_flights_df else pd.DataFrame()

# Calling the function
flights_df = get_flight_data_for_airports(airports_df.copy())

# Change ScheduledArrival(Local) & ActualArrival(Local) to datetime
# Quick fix: Remove the last 6 characters (time zone information) to bypass the Pandas FutureWarning with mixed utc timezones.
flights_df['ScheduledArrival'] = flights_df['ScheduledArrival'].str[:-6]
flights_df['ActualArrival'] = flights_df['ActualArrival'].str[:-6]
flights_df['ScheduledArrival'] = pd.to_datetime(flights_df['ScheduledArrival'])
flights_df['ActualArrival'] = pd.to_datetime(flights_df['ActualArrival'])
flights_df

No flights for Airport: LCY. Skipped.
No flights for Airport: KRH. Skipped.
No flights for Airport: TEB. Skipped.
No flights for Airport: CDW. Skipped.


,FlightNumber,Airline,Aircraft,OriginAirport,OriginIATA,Terminal,ScheduledArrival,ActualArrival,Note,AirportName
0,HU 489,Hainan,Airbus A330-300,Beijing,PEK,1,2025-01-26 06:40:00,2025-01-26 06:40:00,Expected,BER
1,QR 79,Qatar,Boeing 787-9,Doha,DOH,1,2025-01-26 06:55:00,2025-01-26 06:55:00,Expected,BER
2,TP 6442,TAP Air Portugal,Airbus A220-300,Tallinn,TLL,1,2025-01-26 07:50:00,2025-01-26 07:50:00,Expected,BER
3,BT 821,airBaltic,Airbus A220-300,Tallinn,TLL,1,2025-01-26 07:50:00,2025-01-26 07:50:00,Expected,BER
4,A3 3239,Aegean,Airbus A220-300,Tallinn,TLL,1,2025-01-26 07:50:00,2025-01-26 07:50:00,Expected,BER
...,...,...,...,...,...,...,...,...,...,...
1437,DL 5720,Delta Air Lines,Embraer 175,Boston,BOS,4,2025-01-26 11:34:00,NaT,Expected,JFK
1438,DL 47,Delta Air Lines,Airbus A330-900,Amsterdam,AMS,4,2025-01-26 11:38:00,NaT,Expected,JFK
1439,AA 103,American,Boeing 777-200,London,LHR,8,2025-01-26 11:40:00,NaT,Expected,JFK
1440,DL 1411,Delta Air Lines,Airbus A320,Austin,AUS,4,2025-01-26 11:54:00,NaT,Expected,JFK


## Data to MySQL

In [9]:
def connect_to_database(config):
	"""
	Establishes a connection to the MySQL database using the provided configuration from config.yml.
	Args:
			config (dict): Configuration dictionary containing connection details.
	Returns:
			sqlalchemy.engine.Engine: A SQLAlchemy connection engine.
	"""
	if config:
		mysql_config = config.get("mysql", {})
		schema = mysql_config.get("schema", "default_schema")
		host = mysql_config.get("host", "localhost")
		user = mysql_config.get("user", "root")
		password = mysql_config.get("password", "")
		port = mysql_config.get("port", 3306)
		connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
		print("Connection to the database has been established")
		return create_engine(connection_string)
	else:
		raise ValueError("The configuration could not be loaded.")

In [3]:
# Import der Funktionen aus der .py Datei
from database import connect_to_database

In [10]:
engine = connect_to_database(config)

Connection to the database has been established


In [11]:
def update_countries_table(cities_df, engine):
	"""
	Updates the 'countries' table in the database by appending new countries from the given cities DataFrame.
	Args:
			cities_df (pd.DataFrame): DataFrame containing city data, including the 'country' column.
			engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
	"""
	# Extract unique countries
	countries_unique = cities_df["country"].drop_duplicates()
	country_table = pd.DataFrame({"country": countries_unique})
	existing_countries = pd.read_sql("countries", con=engine) # Load existing countries from the database
	new_countries = country_table[~country_table['country'].isin(existing_countries['country'])] # Filter out countries that already exist
	new_countries.to_sql('countries', con=engine, if_exists='append', index=False) # Append new countries to the database
	print(f"Updated 'countries' table with {len(new_countries)} new entries.")

def update_cities_table(cities_df, engine):
	"""
	Updates the 'cities' table in the database by appending new cities from the given cities DataFrame.
	Args:
			cities_df (pd.DataFrame): DataFrame containing city data, including 'country', 'city', and geographic columns.
			engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
	"""
	country_from_sql = pd.read_sql("countries", con=engine) # Load the 'countries' table from the database
	city_table = cities_df.merge(country_from_sql, on="country", how="left") # Merge cities with country data from the database
	existing_cities = pd.read_sql("cities", con=engine) # Load existing cities from the database
	new_cities = city_table[~city_table['city'].isin(existing_cities['city'])] # Filter out cities that already exist
	new_cities = new_cities.drop(columns=["country", "population", "population_year"]) # Drop unnecessary columns and reorder
	new_cities = new_cities[["city", "country_id", "latitude", "longitude"]]
	new_cities.to_sql('cities', con=engine, if_exists='append', index=False) # Append new cities to the database
	print(f"Updated 'cities' table with {len(new_cities)} new entries.")

def update_populations_table(cities_df, engine):
	"""
	Updates the 'populations' table in the database by appending new population data from the given cities DataFrame.
	Args:
			cities_df (pd.DataFrame): DataFrame containing city data, including 'city', 'population', and 'population_year'.
			engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
	"""
	cities_from_sql = pd.read_sql("cities", con=engine) 	# Load the 'cities' table from the database
	population_table = cities_from_sql.merge(cities_df, on="city", how="left") 	# Merge the cities DataFrame with population data
	existing_populations = pd.read_sql("populations", con=engine) 	# Load existing population data from the database
	# Filter out population entries that already exist
	new_populations = population_table[~((population_table['city_id'].isin(existing_populations['city_id'])) & (population_table['population_year'].isin(existing_populations['population_year'])))]
	new_populations = new_populations.drop(columns=['city', 'latitude_x', 'longitude_x', 'country_id', 'country', 'latitude_y', 'longitude_y']) # Drop unnecessary columns
	new_populations.to_sql('populations', con=engine, if_exists='append', index=False) # Append new population data to the database
	print(f"Updated 'populations' table with {len(new_populations)} new entries.")

def update_weather_table(weather_df, engine):
	"""
	Updates the 'weather' table in the database by appending new weather data from the given DataFrame.
	Args:
			weather_df (pd.DataFrame): DataFrame containing weather data, including 'city' and weather details.
			engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
	"""
	cities_from_sql = pd.read_sql("cities", con=engine) # Load the 'cities' table from the database
	weather_table = weather_df.merge(cities_from_sql, on="city", how="left") # Merge the weather DataFrame with city data
	weather_table = weather_table.drop(columns=["city", "country_id", "latitude", "longitude"]) # Drop unnecessary columns
	weather_table = weather_table.dropna(subset=["city_id"]) # Drop rows with missing city IDs (no matching city found)
	existing_weather = pd.read_sql("weather", con=engine) # Load existing weather data from the database
	new_weather = weather_table[~weather_table['city_id'].isin(existing_weather['city_id'])] # Filter out weather entries that already exist
	# Append new weather data to the database
	new_weather.to_sql('weather', con=engine, if_exists='append', index=False)
	print(f"Updated 'weather' table with {len(new_weather)} new entries.")

def update_airports_table(airports_df, engine):
	"""
	Updates the 'airports' table in the database by appending new airport data from the given DataFrame.
	Args:
			airports_df (pd.DataFrame): DataFrame containing airport data, including 'city', 'icao', and location details.
			engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
	"""
	cities_from_sql = pd.read_sql("cities", con=engine) # Load the 'cities' table from the database
	airports_table = airports_df.merge(cities_from_sql, on="city", how="left") # Merge the airports DataFrame with city data
	airports_table = airports_table.drop(columns=["city", "latitude", "longitude", "country_id"])  # Drop unnecessary columns
	airports_table = airports_table.rename(columns={"name": "airportName", "location.lat": "latitude", "location.lon": "longitude"}) # Rename columns for clarity
	airports_table = airports_table.drop_duplicates(subset=["icao"]) # Drop duplicate airports based on 'icao'
	existing_airports = pd.read_sql("SELECT icao FROM airports", con=engine) # Load existing airports from the database
	new_airports = airports_table[~airports_table['icao'].isin(existing_airports['icao'])] # Filter out airports that already exist
	new_airports.to_sql('airports', con=engine, if_exists='append', index=False) # Append new airports to the database
	print(f"Updated 'airports' table with {len(new_airports)} new entries.")

def update_city_airport_table(airports_df, engine):
	"""
	Updates the 'airport_city' table in the database by appending new city-airport relationships.
	Args:
			airports_df (pd.DataFrame): DataFrame containing airport data with associated city information.
			engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
	"""
	cities_from_sql = pd.read_sql("SELECT city_id FROM cities", con=engine) # Load 'cities' table from the database    
	airports_from_sql = pd.read_sql("SELECT airport_id, city_id FROM airports", con=engine) # Load 'airports' table from the database 
	city_airport_table = airports_from_sql.merge(cities_from_sql, on="city_id", how="inner") # Merge airports with city data
	city_airport_table = city_airport_table.dropna(subset=["city_id", "airport_id"]) # Drop rows with missing city_id or airport_id
	existing_city_airport = pd.read_sql("SELECT city_id, airport_id FROM airport_city", con=engine) # Load existing city_airport relationships
	# Filter out existing relationships
	# new_city_airport = city_airport_table.set_index(['city_id', 'airport_id'])[~city_airport_table.set_index(['city_id', 'airport_id']).index.isin(existing_city_airport.set_index(['city_id', 'airport_id']).index)]
	new_city_airport = city_airport_table[~city_airport_table.set_index(['city_id', 'airport_id']).index.isin(existing_city_airport.set_index(['city_id', 'airport_id']).index)]
	# Ensure columns 'city_id' and 'airport_id' are present
	new_city_airport.reset_index(drop=True, inplace=True)
	new_city_airport.to_sql('airport_city', con=engine, if_exists='append', index=False)
	print(f"Updated 'airport_city' table with {len(new_city_airport)} new entries.")

def update_flights_table(flights_df, engine):
	"""
	Updates the 'flights' table in the database by appending new flight data.
	Args:
			flights_df (pd.DataFrame): DataFrame containing flight data, including 'AirportName', 'FlightNumber', and schedule details.
			engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
	"""
	airports_from_sql = pd.read_sql("airports", con=engine) # Load 'airports' table from the database    
	flights_table = airports_from_sql.merge(flights_df, left_on="iata", right_on="AirportName", how="inner") # Merge airports with flights data
	# Drop unnecessary columns
	flights_table = flights_table.drop(columns=['icao', 'iata', 'timeZone', 'airportName', 'shortName', 'municipalityName', 'countryCode', 'latitude', 'longitude', 'city_id', 'AirportName'])
	existing_flights = pd.read_sql("flights", con=engine) # Load existing flights from the database
	# Filter out existing flight entries
	new_flights = flights_table[~((flights_table['FlightNumber'].isin(existing_flights['FlightNumber'])) & (flights_table['ScheduledArrival'].isin(existing_flights['ScheduledArrival'])))]
	new_flights.to_sql('flights', con=engine, if_exists='replace', index=False) # Append new flight data to the database
	print(f"Updated 'flights' table with {len(new_flights)} new entries.")

In [15]:
# Function for updating all tables at one time
def update_all_tables(cities_df, airports_df, flights_df, weather_df, engine):
    """
    Updates all necessary database tables with the provided data.

    Args:
        cities_df (pd.DataFrame): DataFrame containing city data.
        airports_df (pd.DataFrame): DataFrame containing airport data.
        flights_df (pd.DataFrame): DataFrame containing flight data.
        weather_df (pd.DataFrame): DataFrame containing weather data.
        engine (sqlalchemy.engine.Engine): SQLAlchemy engine for database connection.
    """
    # Update 'countries' table
    update_countries_table(cities_df, engine)

    # Update 'cities' table
    update_cities_table(cities_df, engine)

    # Update 'populations' table
    update_populations_table(cities_df, engine)

    # Update 'weather' table
    update_weather_table(weather_df, engine)

    # Update 'airports' table
    update_airports_table(airports_df, engine)

    # Update 'city_airport' table
    update_city_airport_table(airports_df, engine)

    # Update 'flights' table
    update_flights_table(flights_df, engine)

    print("All tables have been successfully updated.")

In [12]:
update_countries_table(cities_df, engine)
update_cities_table(cities_df, engine)
update_populations_table(cities_df, engine)
update_weather_table(weather_df, engine)
update_airports_table(airports_df, engine)
update_city_airport_table(airports_df, engine)
update_flights_table(flights_df, engine)

Updated 'countries' table with 0 new entries.
Updated 'cities' table with 0 new entries.
Updated 'populations' table with 7 new entries.
Updated 'weather' table with 0 new entries.
Updated 'airports' table with 0 new entries.
Updated 'airport_city' table with 0 new entries.
Updated 'flights' table with 1442 new entries.


In [ ]:
update_all_tables(cities_df, airports_df, flights_df, weather_df, engine)